## Getting access to data

In [ ]:
from stravalib import Client
import pickle
import time
import pandas as pd

client = Client()

MY_STRAVA_CLIENT_ID, MY_STRAVA_CLIENT_SECRET = open('access/client.secret').read().strip().split(',')
print ('Client ID and secret read from file'.format(MY_STRAVA_CLIENT_ID) )

In [ ]:
with open('access/access_token.pickle', 'rb') as f:
    access_token = pickle.load(f)
    
print('Latest access token read from file:')
access_token

In [ ]:
if time.time() > access_token['expires_at']:
    print('Token has expired, will refresh')
    refresh_response = client.refresh_access_token(client_id=MY_STRAVA_CLIENT_ID, 
                                               client_secret=MY_STRAVA_CLIENT_SECRET, 
                                               refresh_token=access_token['refresh_token'])
    access_token = refresh_response
    with open('access/access_token.pickle', 'wb') as f:
        pickle.dump(refresh_response, f)
    print('Refreshed token saved to file')

    client.access_token = refresh_response['access_token']
    client.refresh_token = refresh_response['refresh_token']
    client.token_expires_at = refresh_response['expires_at']
        
else:
    print('Token still valid, expires at {}'
          .format(time.strftime("%a, %d %b %Y %H:%M:%S %Z", time.localtime(access_token['expires_at']))))

    client.access_token = access_token['access_token']
    client.refresh_token = access_token['refresh_token']
    client.token_expires_at = access_token['expires_at']


In [ ]:
athlete = client.get_athlete()
print("Athlete's name is {} {}, based in {}, {}".format(athlete.firstname, athlete.lastname, athlete.city, athlete.country))

## Analysing the activities

In [ ]:
activities = client.get_activities(limit=20)
list(activities)[0:10]

In [ ]:
# Look at one activity to get a view of what's available

# sample = list(activities)[0]
# sample.to_dict()

In [ ]:

# Choose some fields of interest from this data in order to read into a DataFrame

my_cols =['name',
          'start_date_local',
          'type',
          'distance',
          'moving_time',
          'elapsed_time',
          'total_elevation_gain',
          'elev_high',
          'elev_low',
          'average_speed',
          'max_speed',
          'average_heartrate',
          'max_heartrate',
          'start_latitude',
          'start_longitude']

In [ ]:
data = []
for activity in activities:
    my_dict = activity.to_dict()
    data.append([activity.id]+[my_dict.get(x) for x in my_cols])
    
# Add id to the beginning of the columns, used when selecting a specific activity
my_cols.insert(0,'id')

In [ ]:
df = pd.DataFrame(data, columns=my_cols)
df

In [ ]:
df.dtypes